[<img src="../header.svg">](../index.ipynd)

---

# Solving a diffusion problem in 3D

Governing equations:
$$ \frac{\partial u}{\partial t} + \nabla \cdot [-D \nabla u] = f
$$
plus boundary and initial condiitions.
## Load utilities


In [1]:
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... done                 *
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/d3f/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Limex/lua/lua-include.lua
LIMEX-Plugin: Loading util.limex.*
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
RepoTrend-Plugin: Extended package.path:./?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;/usr/local/lib/lua/5.1/?.lua;/usr/local/lib/lua/5.1/?/init.lua;/Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/?.lua


In [2]:
ARGS ={
    gridName = "brom-dose2.ugx",
    numRefs = 2,
    endTime = 1.0,
    dt = 0.0125,
}

In [3]:
print(ARGS.gridName)

brom-dose2.ugx


## Initialize UG4 (for 3D and standard algebra)

In [4]:
InitUG(3, AlgebraType("CPU", 1));

## Create Domain

In [5]:
requiredSubsets = {"INNER", "WALL", "IN"}

In [6]:
dom = util.CreateDomain(ARGS.gridName, ARGS.numRefs, requiredSubsets)

Loading Domain brom-dose2.ugx ... done.
Performing integrity check on domain ... done.
Refining(3): 1 2 3 done.


### Create Approximation space

In [7]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [8]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |     0: INNER |      1: WALL |       2: TOP |        3: IN |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |         3471 |         1933 |         1402 |          121 |           15 |
| (lev,    1)    |        24112 |        17966 |         5668 |          433 |           45 |
| (lev,    2)    |       178686 |       154108 |        22792 |         1633 |          153 |
| (lev,    3)    |      1373434 |      1275128 |        91408 |         6337 |          561 |
| (lev,    0, g) |         3471 |         1933 |         140

## Create a Convection-Diffusion-Equation

In [9]:
elemDisc = ConvectionDiffusion("c", "INNER", "fv1")
elemDisc:set_diffusion(10)

In [10]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(0.0, "c", "IN") -- "Offene Dose"
dirichletBnd:add(1.0, "c", "TOP") -- "Zufluss"

In [11]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBnd)

## Create Multigrid solver

In [12]:
solverDesc = 
{
    type = "bicgstab",
    precond = 
    {
        type = "gmg",
        approxSpace= approxSpace,
        smoother= "ilu",
        baseSolver	= "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)


ILU: please use 'set_ordering_algorithm(..)' in the future


## Solve transient problem

In [13]:
function InitialValue(x,y,z,t,si) 
    if (z<1.75) then return 0.0 else return 1.0 end
end

In [14]:
u = GridFunction(approxSpace)
u:set(0.0)
Interpolate("InitialValue", u, "c")

In [15]:
local startTime = 0.0
util.SolveLinearTimeProblem(u, domainDisc, solver, VTKOutput(), "vtk/sol_brom",
                            "ImplEuler", 1, startTime, ARGS.endTime, ARGS.dt); 

SolveLinearTimeProblem, Linear Solver setup:
BiCGStab( restart = 0, min_orthogonality = 0)
 Convergence Check: StdConvCheck( max steps = 100, min defect = 1e-12, relative reduction = 1e-06)
 Preconditioner: 
 | GeometricMultigrid (V-Cycle)
 |  Smoother (3x pre, 3x post): ILU( damping = ConstantDamping(1))
 |  Basesolver ( Baselevel = 0, gathered base = false): AgglomeratingSolver: SuperLU

>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 0.0125
++++++ Assembling Matrix/Rhs for step size 0.0125

   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    4.499741e-01      -------
   %    1:    1.313157e-03    2.918294e-03
   %    2:    1.271353e-05    9.681657e-03
   %    3:    2.147393e-07    1.689061e-02
   % Relative reduction 1.000000e-06 reached after 3 steps.
   % Average reduction over 3 steps: 7.814623e-03
   % %%%%%  Iter